In [13]:
from typing import List, Tuple
import chess
import chess.pgn

In [14]:


def is_fork(fen, last_move):
    game = game_from_fen(fen, last_move)
    nb = 0
    for _, square in attacked_opponent_squares(
        game.board(), game.end().move.to_square, not game.board().turn
    ):
        print(square)
        if is_square_attacked_more_than_defended(game.board(), square, game.board().turn):
            nb += 1
    return nb > 1

def game_from_fen(fen, last_move):
    board = chess.Board(fen)
    game = chess.pgn.Game.from_board(board)
    game.add_main_variation(chess.Move.from_uci(last_move))
    return game



def attacked_opponent_squares(board: chess.Board, from_square: chess.Square, pov: chess.Color) -> List[Tuple[chess.Piece, chess.Square]]:
    pieces = []
    piece = board.piece_at(from_square)
    
    # Get direct attacks first
    direct_squares = board.attacks(from_square)
    for attacked_square in direct_squares:
        attacked_piece = board.piece_at(attacked_square)
        if attacked_piece and attacked_piece.color != pov:
            pieces.append((attacked_piece, attacked_square))
    
    # Check for x-ray attacks if it's a sliding piece
    if piece and piece.piece_type in [chess.BISHOP, chess.ROOK, chess.QUEEN]:
        # Check for x-ray attacks through blocking pieces
        for attacked_square in direct_squares:
            blocking_piece = board.piece_at(attacked_square)
            if blocking_piece and blocking_piece.color == pov:
                # Create a copy of the board and remove the blocking piece
                board_copy = board.copy()
                board_copy.remove_piece_at(attacked_square)
                
                # Get attacks from the original square on the modified board
                xray_squares = board_copy.attacks(from_square)
                for xray_square in xray_squares:
                    xray_piece = board.piece_at(xray_square)
                    if xray_piece and xray_piece.color != pov and (xray_piece, xray_square) not in pieces:
                        pieces.append((xray_piece, xray_square))
                    
    return pieces

In [15]:
fen = "rn1qkb1r/ppp2ppp/5n2/4p3/2B1P3/5Q2/PPP2PPP/RNB1K2R w KQkq - 2 7"
last_move = "f3b3" # In uci is f3b3

In [18]:
is_fork(fen, last_move)

AssertionError: 